In [1]:
import tensorflow as tf
from keras.src.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models, Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet50 import preprocess_input
import pandas as pd
import numpy as np


In [2]:
def create_species_extinction_map():
    df = pd.read_csv('C:/Users/Natalia/Desktop/PADIA/endansys/etiquetas_animales.csv')
    extinction_map = df.groupby('species')['status'].first().to_dict()
    return extinction_map

print(create_species_extinction_map())

{'aguila arpia': 1, 'anaconda': 0, 'armadillo gigante': 0, 'boa constrictor': 0, 'buho cornudo': 0, 'cacique candela': 0, 'caiman aguja': 0, 'capibara': 0, 'caracara': 0, 'colibri esmeralda': 0, 'condor de los andes': 1, 'cusumbo': 0, 'delfin rosado': 1, 'flamenco': 0, 'gallito de roca': 0, 'garza blanca': 0, 'guacamaya azulamarilla': 1, 'guacamaya roja': 1, 'guatin': 0, 'hormiguero gigante': 0, 'iguana verde': 0, 'jaguar': 1, 'jaguarundi': 0, 'lagarto azul': 0, 'loro orejiamarillo': 0, 'manati del caribe': 1, 'marimonda': 0, 'martin pescador': 0, 'mico titi gris': 1, 'mono aullador': 0, 'mono lanudo': 0, 'mono nocturno': 0, 'musarana de cola corta colombiana': 0, 'neque': 0, 'nutria gigante': 1, 'oso andino': 0, 'oso de anteojos': 1, 'paujil': 0, 'pecari de collar': 0, 'perezoso de dos dedos': 0, 'perico carisucio': 0, 'perro de monte': 0, 'puma': 0, 'rana de cristal': 0, 'rana dorada': 1, 'rana venenosa azul': 0, 'tapaculo': 0, 'tapir amazonico': 1, 'tigrillo': 1, 'tinamu': 0, 'titi 

In [3]:
img_size = (224, 224)
batch_size = 32

train_ds = image_dataset_from_directory(
    'C:/Users/Natalia/Desktop/PADIA/endansys/dataset/training',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    'C:/Users/Natalia/Desktop/PADIA/endansys/dataset/validation',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_ds = image_dataset_from_directory(
    'C:/Users/Natalia/Desktop/PADIA/endansys/dataset/testing',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.1, 0.1),
])

base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
base_model.trainable = False

model = models.Sequential([
    data_augmentation,
    layers.Resizing(224, 224),
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_ds.class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(
    patience=10,
    restore_best_weights=True
)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=25,
          callbacks=[early_stopping])

val_loss, val_acc = model.evaluate(val_ds)
print(f"Precisión en validación: {val_acc:.2%}")

test_loss, test_acc = model.evaluate(test_ds)
print(f"Precisión en test: {test_acc:.2%}")

#Aqui empieza el fine-tuning
base_model.trainable = True
fine_tune = 140

for layer in base_model.layers[:fine_tune]:
    layer.trainable = False

#Recompilar con un learning mas bajo para el fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(0.000001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Entrenamiento fine-tuning
model.fit(train_ds,
          validation_data=val_ds,
          epochs=20,
          callbacks=[early_stopping])

val_loss, val_acc = model.evaluate(val_ds)
print(f"Precisión en validación con fine-tuning: {val_acc:.2%}")

test_loss, test_acc = model.evaluate(test_ds)
print(f"Precisión en test con fine-tuning: {test_acc:.2%}")

Found 1383 files belonging to 56 classes.
Found 272 files belonging to 56 classes.
Found 359 files belonging to 56 classes.
Epoch 1/25

44/44 ━━━━━━━━━━━━━━━━━━━━ 33s 688ms/step - accuracy: 0.0517 - loss: 4.2729 - val_accuracy: 0.3051 - val_loss: 3.1562
Epoch 2/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 29s 657ms/step - accuracy: 0.1975 - loss: 3.2927 - val_accuracy: 0.4853 - val_loss: 2.3481
Epoch 3/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 673ms/step - accuracy: 0.3136 - loss: 2.7306 - val_accuracy: 0.6213 - val_loss: 1.7216
Epoch 4/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 670ms/step - accuracy: 0.3841 - loss: 2.3484 - val_accuracy: 0.6875 - val_loss: 1.3831
Epoch 5/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 38s 853ms/step - accuracy: 0.4277 - loss: 2.0766 - val_accuracy: 0.7206 - val_loss: 1.2074
Epoch 6/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 33s 755ms/step - accuracy: 0.4995 - loss: 1.8413 - val_accuracy: 0.7353 - val_loss: 1.0290
Epoch 7/25
44/44 ━━━━━━━━━━━━━━━━━━━━ 32s 718ms/step - accuracy: 0.5291 - loss: 1.6454 - val_accuracy

In [ ]:
import json
class_names_modified = [
    name.lower()
    .replace("_peligro", "")
    .replace("_", " ")
    .strip()
    for name in train_ds.class_names
]

In [44]:
model.save("animal_species_classifier.h5")

TypeError: cannot pickle 'module' object

In [38]:
import tensorflow as tf
model = tf.keras.models.load_model("animal_species_classifier.h5",
                                   custom_objects={'preprocess_input': preprocess_input})

ValueError: No model config found in the file at animal_species_classifier.h5.

In [10]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np

img_path = "C:/Users/Natalia/Desktop/PADIA/endansys/dataset/testing/Buho_cornudo/32.jpg"  # Cambia esto por la ruta de tu imagen
img_size = (224, 224)

# Cargar imagen y redimensionar
img = load_img(img_path, target_size=img_size)

# Convertir a array y escalar
img_array = img_to_array(img)
img_array = preprocess_input(img_array)  # Normalizar igual que en el modelo

# Expandir dimensiones para simular un batch de tamaño 1
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
class_names = train_ds.class_names

predicted_class = np.argmax(predictions[0])
confidence = 100 * np.max(predictions[0])

print(f"La imagen probablemente pertenece a '{class_names_modified[predicted_class]}' con una confianza de {confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
La imagen probablemente pertenece a 'buho cornudo' con una confianza de 45.90%


In [11]:
especies = create_species_extinction_map()
if especies[class_names_modified[predicted_class]] == '1':
    print(f'El animal {class_names_modified[predicted_class]} está en peligro de extinción')
else:
    print(f'El animal {class_names_modified[predicted_class]} no está en peligro de extinción')

El animal buho cornudo no está en peligro de extinción
